In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

import WUBRG
import consts
from LineColors import LineColors
from FormatMetadata import SETS, FORMATS, SET_CONFIG
from JSONHandler import JSONHandler
from FormatMetadata import FormatMetadata
from RawDataFetcher import RawDataFetcher
from RawDataHandler import RawDataHandler
from FramedData import FramedData

# Objects

In [ ]:
class FramedDataFuncs:   
    def __init__(self, SET, FORMAT):
        self._SET = SET
        self._FORMAT = FORMAT
        self._DATA = FramedData(SET, FORMAT)
    
    @property
    def SET(self):
        """The draft set."""
        return self._SET
    
    @property
    def FORMAT(self):
        """The format type."""
        return self._FORMAT
    
    @property
    def DATA(self):
        """The object which contains the data about the set and format."""
        return self._DATA
    
    def deck_group_frame(self, name=None, date=None, summary=False):
        """Returns a subset of the 'GROUPED_ARCHTYPE' data as a DataFrame."""
        return self.DATA.deck_group_frame(name, date, summary)
     
    def deck_archetype_frame(self, deck_color=None, date=None, summary=False):
        """Returns a subset of the 'SINGLE_ARCHTYPE' data as a DataFrame."""
        return self.DATA.deck_archetype_frame(deck_color, date, summary)
    
    def card_frame(self, name=None, deck_color=None, date=None, card_color=None, card_rarity=None, summary=False):
        """Returns a subset of the 'CARD' data as a DataFrame."""
        return self.DATA.card_frame(name, deck_color, date, card_color, card_rarity, summary)
        
    
    #Helper Functions
    def get_avg_winrate(self, day=None, arch='All Decks'):
        if day: return self.deck_group_frame(date=day, summary=False).loc[(day, arch)]['Win %']
        else: return self.deck_group_frame(date=day, summary=True).loc[arch]['Win %'] 
            
    def get_games_played(self, deck_color):
        if deck_color: return self.deck_archetype_frame(deck_color=deck_color, summary=True)['Games'].sum()
        else: return self.deck_group_frame(name='All Decks', summary=True)['Games']
    
    
    def graph_pick_stats(self, card_name, roll=1):
        taken_data = self.card_frame(name=card_name, deck_color='')[['ALSA', 'ATA']]
        taken_data.index = [tup[0][5:] for tup in taken_data.index]
        rolling = taken_data.rolling(window=roll, min_periods=1, center=True).mean()

        mx = min(max(taken_data.max()) + 0.25, 15) 
        mn = max(min(taken_data.min()) - 0.25, 1)
        tit_str = f"{self.SET} - {self.FORMAT}\n{card_name}"

        rolling.plot(ylim=(mx, mn), grid=True, title=tit_str)

        return taken_data

    def compare_card_evaluations(self, start_date, end_date):
        def inner_func(date):
            df = self.card_frame(date=date, deck_color='')
            df.index = [tup[2] for tup in df.index]
            return df

        first = inner_func(date='2022-02-10')
        last = inner_func(date='2022-02-15')
        diff = last[['ALSA', 'ATA', 'Color', 'Rarity']].copy()
        diff['Δ ALSA'] = first['ALSA'] - last['ALSA']
        diff['Δ ATA'] = first['ATA'] - last['ATA']
        return diff[['ALSA', 'Δ ALSA', 'ATA', 'Δ ATA', 'Color', 'Rarity']]

    
    def get_card_summary(self, card_name, colors='', roll=1):
        frame = self.card_frame(name=card_name, deck_color=colors)[['GIH WR', 'ALSA', '# GP', 'IWD']]
        frame.index = [tup[0][5:] for tup in frame.index]
        rolling = frame.rolling(window=roll, min_periods=1, center=True).mean()
        return rolling

    def plot_card_summary(self, card_name, colors='', roll=1):
        rolling = self.get_card_summary(card_name, colors, roll)
        title = f"{card_name}"

        col_filt = f"Color Filter: {colors}"
        rol_filt = f"Rolling Average: {roll} Days"

        if colors and roll > 1:
            title += f"\n{col_filt}  -  {rol_filt}"
        elif colors:
                title += f"\n{col_filt}"
        elif roll > 1:
                title += f"\n{rol_filt}"

        rolling.plot(subplots=True, layout=(2,2), figsize=(12,8), title=title)
        

    def get_top(self, column, count=10, asc=True, card_color=None, card_rarity=None, deck_color='', play_lim=None):
        frame = self.card_frame(deck_color=deck_color, summary=True, card_rarity=card_rarity)
        frame = frame.sort_values(column, ascending=asc)

        if card_color is not None:
            card_color = WUBRG.get_color_identity(card_color)
            frame = frame[frame['Color'] == card_color]

        # TODO: Implement filtering here.
        if play_lim is not None:
            if type(play_lim) is float: play_lim *= self.get_games_played(deck_color)
            print(f'Minimum Games played to be included: {play_lim}')
            frame = frame[frame['# GP'] >= play_lim]

        return frame.head(count)

    
    def get_archetype_frame(self, colors, roll=1):
        win_rate_frame = self.deck_archetype_frame(deck_color=colors)
        win_rate_frame.index = [tup[0] for tup in win_rate_frame.index]
        #win_rate_frame = win_rate_frame[['Splash', 'Games', 'Win %']]
        win_rate_frame = win_rate_frame[win_rate_frame['Splash'] == False][['Wins', 'Games']]
        rolling = win_rate_frame.rolling(window=roll, min_periods=1, center=True).mean().round()
        rolling['Win %'] = round((rolling['Wins'] / rolling['Games']) * 100, 2)
        rolling['Avg. Win%'] = [self.get_avg_winrate(idx) for idx in win_rate_frame.index]
        rolling['2C Win%'] = [self.get_avg_winrate(idx, arch='Two-color') for idx in win_rate_frame.index]
        rolling['Win % Offset'] = rolling['Win %'] - rolling['Avg. Win%']
        return rolling

    def get_archetype_winrate_history(self, color_filter=None, roll=1):
        d = dict()
        for col in WUBRG.COLOR_PAIRS:
            temp_frame = self.get_archetype_frame(col)
            d[col] = temp_frame['Win %']
        d['AVG'] = temp_frame ['Avg. Win%']
        d['2C'] = temp_frame ['2C Win%']

        test_frame = pd.DataFrame.from_dict(d)
        test_frame.index = [idx[5:] for idx in test_frame.index]
        if color_filter:
            col_filt = [col for col in WUBRG.COLOR_PAIRS if color_filter in col] + ['AVG', '2C'] 
            test_frame = test_frame[col_filt]

        rolling = test_frame.rolling(window=roll, min_periods=1, center=True).mean()
        return rolling

    def plot_archetype_winrate_history(self, color_filter=None, roll=1):
        test_frame = self.get_archetype_winrate_history(color_filter, roll)
        lc = LineColors()
        title = "Archetpye Winrates"

        col_filt = f"Color Filter: {color_filter}"
        rol_filt = f"Rolling Average: {roll} Days"

        if color_filter and roll > 1:
            title += f"\n{col_filt}  -  {rol_filt}"
        elif color_filter:
                title += f"\n{col_filt}"
        elif roll > 1:
                title += f"\n{rol_filt}"
        test_frame.plot(figsize=(20, 10), color=lc.get_col_array(color_filter), title=title, lw=2.5, grid=True)
        
        
    def get_archetype_playrate_history(self, color_filter=None, roll=1):    
        d = dict()
        for col in WUBRG.COLOR_PAIRS:
            d[col] = self.get_archetype_frame(col)['Games']

        test_frame = pd.DataFrame.from_dict(d)
        test_frame.index = [idx[5:] for idx in test_frame.index]
        rolling = test_frame.rolling(window=roll, min_periods=1, center=True).mean()
        total = rolling.sum(axis=1)
        playrate = rolling.divide(list(total),axis=0) * 100

        if color_filter:
            col_filt = [col for col in WUBRG.COLOR_PAIRS if color_filter in col]
            playrate = playrate[col_filt]

        return playrate

    def plot_archetype_playrate_history(self, color_filter=None, roll=1):
        test_frame = self.get_archetype_playrate_history(color_filter, roll)
        lc = LineColors()
        title = "Archetpye Playrates"

        col_filt = f"Color Filter: {color_filter}"
        rol_filt = f"Rolling Average: {roll} Days"

        if color_filter and roll > 1:
            title += f"\n{col_filt}  -  {rol_filt}"
        elif color_filter:
                title += f"\n{col_filt}"
        elif roll > 1:
                title += f"\n{rol_filt}"
        test_frame.plot(figsize=(20, 10), color=lc.get_col_array(color_filter), title=title, lw=2.5, grid=True) 

In [ ]:
class WinrateFuncs:   
    def __init__(self, parent):
        self._DATA = parent.DATA
    
    def get_avg_winrate(self, day=None, arch='All Decks'):
        if day: return self._DATA.deck_group_frame(date=day, summary=False).loc[(day, arch)]['Win %']
        else: return self._DATA.deck_group_frame(date=day, summary=True).loc[arch]['Win %']   

    
    def get_archetype_frame(self, colors, roll=1):
        win_rate_frame = self.deck_archetype_frame(deck_color=colors)
        win_rate_frame.index = [tup[0] for tup in win_rate_frame.index]
        #win_rate_frame = win_rate_frame[['Splash', 'Games', 'Win %']]
        win_rate_frame = win_rate_frame[win_rate_frame['Splash'] == False][['Wins', 'Games']]
        rolling = win_rate_frame.rolling(window=roll, min_periods=1, center=True).mean().round()
        rolling['Win %'] = round((rolling['Wins'] / rolling['Games']) * 100, 2)
        rolling['Avg. Win%'] = [self.get_avg_winrate(idx) for idx in win_rate_frame.index]
        rolling['2C Win%'] = [self.get_avg_winrate(idx, arch='Two-color') for idx in win_rate_frame.index]
        rolling['Win % Offset'] = rolling['Win %'] - rolling['Avg. Win%']
        return rolling

    def get_archetype_winrate_history(self, color_filter=None, roll=1):
        d = dict()
        for col in WUBRG.COLOR_PAIRS:
            temp_frame = self.get_archetype_frame(col)
            d[col] = temp_frame['Win %']
        d['AVG'] = temp_frame ['Avg. Win%']
        d['2C'] = temp_frame ['2C Win%']

        test_frame = pd.DataFrame.from_dict(d)
        test_frame.index = [idx[5:] for idx in test_frame.index]
        if color_filter:
            col_filt = [col for col in WUBRG.COLOR_PAIRS if color_filter in col] + ['AVG', '2C'] 
            test_frame = test_frame[col_filt]

        rolling = test_frame.rolling(window=roll, min_periods=1, center=True).mean()
        return rolling

    def plot_archetype_winrate_history(self, color_filter=None, roll=1):
        test_frame = self.get_archetype_winrate_history(color_filter, roll)
        lc = LineColors()
        title = "Archetpye Winrates"

        col_filt = f"Color Filter: {color_filter}"
        rol_filt = f"Rolling Average: {roll} Days"

        if color_filter and roll > 1:
            title += f"\n{col_filt}  -  {rol_filt}"
        elif color_filter:
                title += f"\n{col_filt}"
        elif roll > 1:
                title += f"\n{rol_filt}"
        test_frame.plot(figsize=(20, 10), color=lc.get_col_array(color_filter), title=title, lw=2.5, grid=True)
        
        
    def get_archetype_playrate_history(self, color_filter=None, roll=1):    
        d = dict()
        for col in WUBRG.COLOR_PAIRS:
            d[col] = self.get_archetype_frame(col)['Games']

        test_frame = pd.DataFrame.from_dict(d)
        test_frame.index = [idx[5:] for idx in test_frame.index]
        rolling = test_frame.rolling(window=roll, min_periods=1, center=True).mean()
        total = rolling.sum(axis=1)
        playrate = rolling.divide(list(total),axis=0) * 100

        if color_filter:
            col_filt = [col for col in WUBRG.COLOR_PAIRS if color_filter in col]
            playrate = playrate[col_filt]

        return playrate

    def plot_archetype_playrate_history(self, color_filter=None, roll=1):
        test_frame = self.get_archetype_playrate_history(color_filter, roll)
        lc = LineColors()
        title = "Archetpye Playrates"

        col_filt = f"Color Filter: {color_filter}"
        rol_filt = f"Rolling Average: {roll} Days"

        if color_filter and roll > 1:
            title += f"\n{col_filt}  -  {rol_filt}"
        elif color_filter:
                title += f"\n{col_filt}"
        elif roll > 1:
                title += f"\n{rol_filt}"
        test_frame.plot(figsize=(20, 10), color=lc.get_col_array(color_filter), title=title, lw=2.5, grid=True) 

In [ ]:
class SetManager:
    def __init__(self, SET):        
        self._SET = SET
        self._DATA = {f: FramedDataFuncs(SET, f) for f in FORMATS}
        
    @property
    def SET(self):
        """The draft set."""
        return self._SET
            
    @property
    def BO1(self):
        """Premier Draft data."""
        return self._DATA['PremierDraft']
    
    @property
    def BO3(self):
        """Traditional Draft data."""
        return self._DATA['TradDraft']
    
    @property
    def QD(self):
        """Quick Draft data."""
        return self._DATA['QuickDraft']

# Initialization

In [ ]:
set_data = SetManager('NEO')

## Current Tests

In [ ]:
def test_plot(self):
    color_filter=None
    roll=1
    test_frame = set_data.BO1.get_archetype_playrate_history(color_filter, roll)

    W = (0.85,0.85,0.2,0.8)
    U = (0,0,1,0.8)
    B = (0,0,0,0.8)
    R = (1,0,0,0.8)
    G = (0,0.7,0,0.8)
    P = (1,0,1,0.8)
    LW = (0.85,0.85,0.3,0.5)
    LU = (0,0,1,0.5)
    LB = (0.2,0.2,0.2,0.5)
    LR = (1,0.2,0.2,0.5)
    LG = (0.2,0.7,0.2,0.5)
    LP = (1,0.25,1,0.45)

    line_colors = [W, U, B, R, G, LW, LU, LB, LR, LG, P, LP]



    title = "Archetpye Playrates"

    col_filt = f"Color Filter: {color_filter}"
    rol_filt = f"Rolling Average: {roll} Days"

    if color_filter and roll > 1:
        title += f"\n{col_filt}  -  {rol_filt}"
    elif color_filter:
            title += f"\n{col_filt}"
    elif roll > 1:
            title += f"\n{rol_filt}"
    test_frame.plot(figsize=(20, 10), color=line_colors, title=title, lw=2.5, grid=True)
    
test_plot()

In [ ]:
def new_plot():
    pass
new_plot()

# Data Graphing and Display

## Card Summary

In [ ]:
set_data.BO1.plot_card_summary('Dokuchi Shadow-Walker', 'BG', 3)

In [ ]:
set_data.BO1.plot_card_summary('Norika Yamazaki, the Poet', roll=3)

In [ ]:
set_data.BO1.plot_card_summary('Clawing Torment', roll=3)

In [ ]:
TRGT_CARDS = ['Roaring Earth', 'Tales of Master Seshiro', 'Befriending the Moths']
TRGT_PAIR = ''
ROLL = 3

for CARD in TRGT_CARDS:
    set_data.BO1.plot_card_summary(CARD, colors=TRGT_PAIR, roll=ROLL)

In [ ]:
set_data.BO1.get_top('GIH WR', count=25, asc=False, deck_color='', card_color=None, card_rarity='C', play_lim=0.005)

## Card Pick Order Stats

In [ ]:
set_data.BO1.graph_pick_stats('Imperial Oath', 1)

In [ ]:
set_data.BO1.graph_pick_stats('Behold the Unspeakable', 3)

In [ ]:
to_graph = ['Imperial Oath', 'Behold the Unspeakable', 'Virus Beetle', 'Network Disruptor']
for card in to_graph:
    set_data.BO1.graph_pick_stats(card, 1)

## Archetype Winrate History

In [ ]:
set_data.BO1.get_archetype_winrate_history(roll=1)

In [ ]:
set_data.BO1.get_archetype_winrate_history(roll=3)

In [ ]:
ROLL = 1
SET_DATA = set_data.BO1
SET_DATA.plot_archetype_winrate_history('W', 3)
SET_DATA.plot_archetype_winrate_history('U', 3)
SET_DATA.plot_archetype_winrate_history('B', 3)
SET_DATA.plot_archetype_winrate_history('R', 3)
SET_DATA.plot_archetype_winrate_history('G', 3)
SET_DATA.plot_archetype_winrate_history('', 3)

In [ ]:
ROLL = 1
SET_DATA = set_data.BO3
SET_DATA.plot_archetype_winrate_history('W', 3)
SET_DATA.plot_archetype_winrate_history('U', 3)
SET_DATA.plot_archetype_winrate_history('B', 3)
SET_DATA.plot_archetype_winrate_history('R', 3)
SET_DATA.plot_archetype_winrate_history('G', 3)
SET_DATA.plot_archetype_winrate_history('', 3)

## Archetype Playrate History

In [ ]:
set_data.BO1.get_archetype_playrate_history(roll=1)

In [ ]:
set_data.BO1.get_archetype_playrate_history(roll=3)

In [ ]:
ROLL = 1
SET_DATA = set_data.BO1
SET_DATA.plot_archetype_playrate_history('W', ROLL)
SET_DATA.plot_archetype_playrate_history('U', ROLL)
SET_DATA.plot_archetype_playrate_history('B', ROLL)
SET_DATA.plot_archetype_playrate_history('R', ROLL)
SET_DATA.plot_archetype_playrate_history('G', ROLL)
SET_DATA.plot_archetype_playrate_history('', ROLL)

In [ ]:
ROLL = 1
SET_DATA = set_data.BO3
SET_DATA.plot_archetype_playrate_history('W', ROLL)
SET_DATA.plot_archetype_playrate_history('U', ROLL)
SET_DATA.plot_archetype_playrate_history('B', ROLL)
SET_DATA.plot_archetype_playrate_history('R', ROLL)
SET_DATA.plot_archetype_playrate_history('G', ROLL)
SET_DATA.plot_archetype_playrate_history('', ROLL)

## Card Pick Order Changes

In [ ]:
diff = set_data.BO1.compare_card_evaluations('2022-02-15', '2022-02-17')
commons = diff[diff['Rarity'] == 'C']
uncommons = diff[diff['Rarity'] == 'U']

In [ ]:
SRT_TRG = 'ALSA'
commons.sort_values(SRT_TRG, ascending=False).head(20)

In [ ]:
commons.sort_values(SRT_TRG, ascending=True).head(20)

In [ ]:
uncommons.sort_values(SRT_TRG, ascending=False).head(10)

In [ ]:
uncommons.sort_values(SRT_TRG, ascending=True).head(10)